# 在云器Lakehouse的同一张表中进行向量和标量检索：数据准备

GitHub 提供15 多种事件类型，包括新提交和分叉事件、打开新票证、评论和向项目添加成员。这些事件会汇总到每小时存档中，您可以使用任何 HTTP 客户端从 https://www.gharchive.org/ 上访问这些存档。

## 数据准备过程
- 通过wget从 https://www.gharchive.org/ 将归档的json.gz文件下载到本地，本文下载了2025-01-01一天的24个文件。
- 通过Lakehouse的PUT命令，将数据PUT用户VOLUME(USER VOLUME)上.
- 通过SQL直接读取VOLUMR目录里的文件，然后写入github_event_issuesevent表中。
- 将数据从github_event_issuesevent写入到目标表github_event_issuesevent_embedding中。

## 安装 Zettapark
If installed, please pass this step.

In [1]:
#!pip install -U clickzetta-zettapark-python -i https://pypi.tuna.tsinghua.edu.cn/simple

In [2]:
# Zettapark modules
from clickzetta.zettapark.session import Session
import clickzetta.zettapark.types as T
import logging
logging.getLogger("clickzetta.zettapark").setLevel(logging.ERROR)

In [3]:
# Get a nicer output from .show()
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

## 连接到 Lakehouse

This example is using the [config.json](conifg.json) file to connect to Lakehouse. 

Create a connection to Lakehouse.

This example is using a JSON file with the following structure
```
{
  "username": "Please enter your username",
  "password": "Please enter your password",
  "service": "Please enter your service address, e.g. api.clickzetta.com",
  "instance": "Please enter your instance ID",
  "workspace": "Please enter your workspace, e.g. gharchive",
  "schema": "Please enter your schema, e.g. public",
  "vcluster": "Please enter your virtual cluster, e.g. default_ap",
  "sdk_job_timeout": 10,
  "hints": {
    "sdk.job.timeout": 3,
    "query_tag": "Introduction to Zettapark for Python"
  }
}


```

In [4]:
import json

# read connection parameters
with open('config-vector.json', 'r') as config_file:
    config = json.load(config_file)

print("Connecting to Lakehouse.....\n")

# create session
session = Session.builder.configs(config).create()

print("Connected and context as below...\n")

# print(session.sql("SELECT current_instance_id(), current_workspace(),current_workspace_id(), current_schema(), current_user(),current_user_id(), current_vcluster()").collect())

Connecting to Lakehouse.....

Connected and context as below...



In [12]:
data_path = "./data/gharchive/" # WHere the data files is stored localy
data_volume_name = "user" # Using the user volume
DATABASE_NAME = config['workspace']# Name of Database 
DATABASE_SCHEMA = config['schema'] # Name of schema to store data and files in 
FULLY_QUALIFIED_NAME = f"{DATABASE_NAME}.{DATABASE_SCHEMA}"

Create the schema if they do not already exists

In [6]:
session.get_current_schema()

'PUBLIC'

## 下载 GHArchive data

In [ ]:
# Activity for 01-01 of January 2025
!wget -P ./data/gharchive/  https://data.gharchive.org/2025-01-{01..01}-{0..23}.json.gz

## Put GHArchive data to User Volume

In [13]:
# Upload the downloaded files to user volume, please be care the path of user volume format
putResult = session.file.put(f"{data_path}*.json.gz", f"volume:{data_volume_name}://~/gharchive/json/gz/", auto_compress=False, overwrite=True)

In [ ]:
# List files in user volume
session.sql("LIST USER VOLUME").show(100)

## Transform GHArchive data

In [ ]:
ddl_github_event_issuesevent =
"""
        CREATE TABLE IF NOT EXISTS github_event_issuesevent (
            row_id BIGINT identity, -- 自增列，作为第一列
            event_id BIGINT,
            event_type VARCHAR(255),
            created_at TIMESTAMP,
            partition_date DATE,
            public BOOLEAN,
            repo_id VARCHAR(255),
            repo_name VARCHAR(255),
            repo_url VARCHAR(255),
            actor_id VARCHAR(255),
            actor_login VARCHAR(255),
            actor_gravatar_id VARCHAR(255),
            actor_avatar_url VARCHAR(255),
            actor_url VARCHAR(255),
            org_id VARCHAR(255),
            org_login VARCHAR(255),
            org_gravatar_id VARCHAR(255),
            org_avatar_url VARCHAR(255),
            org_url VARCHAR(255),
            issue_body STRING
        )
        PARTITIONED BY(partition_date);
"""

In [ ]:
session.sql(ddl_github_event_issuesevent).collect()

In [ ]:
ddl_github_event_issuesevent_embedding =
"""
        CREATE TABLE IF NOT EXISTS github_event_issuesevent_embedding (
        row_id BIGINT,
        event_id BIGINT,
        event_type VARCHAR(255),
        created_at TIMESTAMP,
        partition_date DATE,
        public BOOLEAN,
        repo_id VARCHAR(255),
        repo_name VARCHAR(255),
        repo_url VARCHAR(255),
        actor_id VARCHAR(255),
        actor_login VARCHAR(255),
        actor_gravatar_id VARCHAR(255),
        actor_avatar_url VARCHAR(255),
        actor_url VARCHAR(255),
        org_id VARCHAR(255),
        org_login VARCHAR(255),
        org_gravatar_id VARCHAR(255),
        org_avatar_url VARCHAR(255),
        org_url VARCHAR(255),
        issue_body STRING,
        issue_body_embedding vector(float, 1024),
        INDEX index_issue_body_vector (issue_body_embedding) USING VECTOR  PROPERTIES(
          "scalar.type" = "f32",
          "distance.function" = "l2_distance"
        ),
        INDEX index_issue_body_inverted (issue_body) INVERTED  PROPERTIES('analyzer'='unicode'),
    
    )
    PARTITIONED BY(partition_date);
    
    CREATE BLOOMFILTER INDEX index_rowid_bf ON TABLE github_event_issuesevent_embedding(row_id);
"""

In [ ]:
session.sql(ddl_github_event_issuesevent_embedding).collect()

In [49]:
query_clean_and_transform = """
WITH
-- read data from data lake volume
gh_events as
(
    select * from user volume(
    data STRING
    )using csv
    options(
    'header'='false',
    'sep'='\n',
    'timeZone' = 'UTC',
    'encoding' = 'utf-8',
    'compression' = 'gzip',
    'quote'='\0'
    ) SUBDIRECTORY 'gharchive/json/gz'
),
-- convert string to json
json_format_data AS
(
    SELECT JSON_PARSE(data) as json_data FROM gh_events
)
-- clean and transform data, set issue_body_embedding to NULL tobe embedded
INSERT INTO github_event_issuesevent
SELECT    
          json_extract_bigint(json_data, '$.id') AS event_id,
          json_extract_string(json_data, '$.type') AS event_type,
          CONVERT_TIMEZONE('UTC','Asia/Shanghai',cast(json_extract(json_data, '$.created_at') as timestamp)) AS created_at,
          cast(created_at as date) AS partition_date,
          json_extract_boolean(json_data, '$.public') AS public,
          json_extract_string(json_data, '$.repo.id') AS repo_id,
          json_extract_string(json_data, '$.repo.name') AS repo_name,
          json_extract_string(json_data, '$.repo.url') AS repo_url,
          json_extract_string(json_data, '$.actor.id') AS actor_id,
          json_extract_string(json_data, '$.actor.login') AS actor_login,
          json_extract_string(json_data, '$.actor.gravatar_id') AS actor_gravatar_id,
          json_extract_string(json_data, '$.actor.avatar_url') AS actor_avatar_url,
          json_extract_string(json_data, '$.actor.url') AS actor_url,
          json_extract_string(json_data, '$.org.id') AS org_id,
          json_extract_string(json_data, '$.org.login') AS org_login,
          json_extract_string(json_data, '$.org.gravatar_id') AS org_gravatar_id,
          json_extract_string(json_data, '$.org.avatar_url') AS org_avatar_url,
          json_extract_string(json_data, '$.org.url') AS org_url,
          json_extract_string(json_data, '$.payload.issue.body') AS issue_body,
FROM      json_format_data
WHERE     json_extract_string(json_data, '$.type') = "IssuesEvent" AND LENGTH(json_extract_string(json_data, '$.payload.issue.body'))>3;
"""

In [39]:
session.sql(query_clean_and_transform).collect()

[Row(result_message='OPERATION SUCCEED')]

In [ ]:
query_insert_target_table = """
    INSERT INTO github_event_issuesevent_embedding
    SELECT *, NULL FROM  github_event_issuesevent where partition_date='2025-01-01';
"""

In [ ]:
session.sql(query_insert_target_table).collect()

## Clean Files

In [ ]:
session.sql(f"remove user volume SUBDIRECTORY 'gharchive/json/gz'").collect()

In [31]:
session.close()